In [1]:
!python -V

Python 3.8.20


In [3]:
import torch
print(torch.version.cuda)  # Check the CUDA version
print(torch.cuda.is_available())  # Check if CUDA is available

11.8
True


In [ ]:
# pip install -r requirements.txt
# conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia

# Process data

In [5]:
import json
from collections import defaultdict

with open("/home/hoang/DSI-transformers/data/12_7/original/legal_corpus_update.json", "r", encoding="utf-8") as f:
    legal_corpus = json.load(f)
with open("/home/hoang/DSI-transformers/data/12_7/original/train_12x7_retrieval.json", "r", encoding="utf-8") as f:
    train_data = json.load(f)
with open("/home/hoang/DSI-transformers/data/12_7/original/validation_12x7_retrieval.json", "r", encoding="utf-8") as f:
    val_data = json.load(f)
with open("/home/hoang/DSI-transformers/data/12_7/original/test_12x7_retrieval.json", "r", encoding="utf-8") as f:
    test_data = json.load(f)

def process_data(corpus, data, name="train"):
    count = 0
    for laws in corpus:
        for article in laws["articles"]:
            count += 1
    new_data = []
    id = 0
    for laws in corpus:
        for article in laws["articles"]:
            if name != "test":
                new_data.append({
                    "text_id": str(id),
                    "text": f'document: {laws["law_id"]}\n{article["title"]}\n{article["text"]}'
                })
            for question in data["items"]:
                for rel_article in question["relevant_articles"]:
                    if laws["law_id"] == rel_article["law_id"] and article["article_id"] == rel_article["article_id"]:
                        new_data.append({
                            "text_id": str(id),
                            "text": f'question: {question["question_full"]}'
                        })
                        break
            id+=1
            print(f"Processing {name}_data: {'{:.1%}'.format(id/count)}", end='\r')
    return new_data

def save_data(data, name="train"):
    with open(f"/home/hoang/DSI-transformers/data/12_7/processed/{name}.jsonl", "w") as f:
        for line in data:
            json_line = json.dumps(line, ensure_ascii=False)
            f.write(json_line + "\n")
    print(f"Saved to /home/hoang/DSI-transformers/data/12_7/processed/{name}.jsonl")

def group_items_by_text(data):
    # Sử dụng defaultdict để gom nhóm các phần tử có text giống nhau
    result = defaultdict(list)
    for item in data:
        result[item["text"]].append(item["text_id"])

    # Chuyển đổi kết quả thành danh sách dictionary như mong muốn
    grouped_data = [{"text_id": ids, "text": text} for text, ids in result.items()]
    return grouped_data

new_train_data = process_data(legal_corpus, train_data, "train")
new_val_data = process_data(legal_corpus, val_data, "val")
new_test_data = process_data(legal_corpus, test_data, "test")
new_test_data = group_items_by_text(new_test_data)

save_data(new_train_data, "train")
save_data(new_val_data, "val")
save_data(new_test_data, "test")

Saved to /home/hoang/DSI-transformers/data/12_7/processed/train.jsonl
Saved to /home/hoang/DSI-transformers/data/12_7/processed/val.jsonl
Saved to /home/hoang/DSI-transformers/data/12_7/processed/test.jsonl


In [23]:
!nvidia-smi

Thu Nov 14 02:21:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        Off | 00000000:86:00.0 Off |                  Off |
| 38%   64C    P2             370W / 450W |  11351MiB / 24564MiB |    100%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Train model

note: num epoch = ceil(max_steps / num_batch)

https://huggingface.co/google-t5/t5-large

Language(s) (NLP): English, French, Romanian, German

Method used:
- model: t5-large
- indexing: Inputs2Target
- document represent: Direct Indexing (only consider first 32 tokens of each document)
- represent docid for retrieval: Naively Structured String Identifiers

In [ ]:
%cd /home/hoang/DSI-transformers
!CUDA_VISIBLE_DEVICES=0 python train.py \
    --model_name_or_path t5-large \
    --train_data data/12_7/processed/train.jsonl \
    --eval_data data/12_7/processed/val.jsonl \
    --test_data data/12_7/processed/test.jsonl \
    --max_steps 10000 --warmup_steps 500 \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 2 \
    --eval_steps 1000 --save_steps 1000 \
    --wandb_name JP-t5-large


100%|████████████████████████████████████████▉| 421/422 [02:19<00:00,  3.15it/s]
                                                                                
{'eval_loss': 2.295225143432617, 'eval_accuracy': 0.002369668246445498, 'eval_runtime': 141.3435, 'eval_samples_per_second': 5.971, 'eval_steps_per_second': 2.986, 'epoch': 4.28}
100%|█████████████████████████████████████████| 422/422 [02:20<00:00,  3.12it/s]
                                                                                Saving model checkpoint to ./results/checkpoint-9000
Configuration saved in ./results/checkpoint-9000/config.json
Model weights saved in ./results/checkpoint-9000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-9000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-9000/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-7000] due to args.save_total_limit
{'loss': 1.9376, 'learning_rate': 4.846938775510204e-05, 'epoch': 4.31}      

# Evaluate

In [ ]:
%cd /home/hoang/DSI-transformers
!CUDA_VISIBLE_DEVICES=0 python evaluate.py \
    --model_name_or_path results/checkpoint-8000 \
    --test_data data/12_7/processed/test.jsonl \
    --per_device_eval_batch_size 32 \
    --num_beams 200

/home/hoang/DSI-transformers
/home/hoang/.conda/envs/dsi/lib/python3.8/site-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state

In [1]:
%cd /home/hoang/DSI-transformers
!CUDA_VISIBLE_DEVICES=0 python evaluate.py \
    --model_name_or_path results/checkpoint-8000 \
    --test_data data/12_7/processed/test.jsonl \
    --per_device_eval_batch_size 2 \
    --num_beams 200

/home/hoang/DSI-transformers
/home/hoang/.conda/envs/dsi/lib/python3.8/site-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state

Lý giải nguyên nhân kq thấp:
- model t5 không hỗ trợ tiếng nhật
- sử dụng method direct indexing, tức chỉ xem xét 32 token đầu tiên, mà data luật thì phần đầu hầu hết là tên đạo luật, chương, tên chương nên sẽ bị lặp khá nhiều
- document và query trong data là quan hệ n-n, nên sẽ làm model bị mơ hồ khi index